# Tutorial
## Install datatable

Let’s begin by installing the latest stable version of __datatable__ from PyPI:


~~~ bash
$ pip install datatable
~~~

If this didn’t work for you, or if you want to install the bleeding edge version of the library, please check the [Installation page](install.rst)

Assuming the installation was successful, you can now import the library in a JupyterLab notebook or in a Python console:

In [1]:
import datatable as dt
dt.__version__

'0.8.0'

## Create Frame

The fundamental unit of analysis in datatable is a **Frame**. It is the same notion as a pandas DataFrame or SQL table: data arranged in a two-dimensional array with rows and columns.

You can create a **Frame** object from a variety of data sources:

* from a python **list** or **dictionary**:

In [2]:
import datatable as dt
import math

DT = dt.Frame(A=range(5), B=[1.7, 3.4, 0, None, -math.inf],
                 stypes={"A": dt.int64})
DT

,A,B
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,1.7
1,1,3.4
2,2,0
3,3,NA
4,4,−inf


* from a **numpy array**

In [3]:
import numpy as np
np.random.seed(1)
DT2 = dt.Frame(np.random.randn(3))
DT2

,C0
,▪▪▪▪▪▪▪▪
0,1.62435
1,−0.611756
2,−0.528172


* from a **pandas DataFrame**.

In [4]:
import pandas as pd
DT3 = dt.Frame(pd.DataFrame({"A": range(3)}))
DT3

,A
,▪▪▪▪▪▪▪▪
0,0
1,1
2,2


## Convert Frame

Convert an existing Frame into a numpy array, a pandas DataFrame, or a pure Python object:

In [5]:
DT_numpy = DT.to_numpy()
DT_pandas = DT.to_pandas()
DT_list = DT.to_list()
DT_dict = DT.to_dict()
DT_tuple = DT.to_tuples()

## Read data

You can also load a CSV/text/Excel file, or open a previously saved binary **.jay** file:

~~~ python
DT4 = dt.fread("dataset_01.xlsx")
DT5 = dt.fread("dataset_02.csv")
DT6 = dt.open("data.jay")
~~~

**fread ()** function shown above is both powerful and extremely fast. It can automatically detect parse parameters for the majority of text files, load data from .zip archives or URLs, read Excel files, and much more.

* Automatically detects separators, headers, column types, quoting rules, etc.
* Reads from majority of text files, load data from .zip archives or URLs, read Excel files, URL, shell, raw text, * archives, glob
* Provides multi-threaded file reading for maximum speed
* Includes a progress indicator when reading large files
* Reads both RFC4180-compliant and non-compliant files

## Write data

Write the Frame’s content into a csv file in a multi-threaded way:

~~~ python

DT.to_csv("out.csv")
~~~

## Frame Properties

Investigate your Frame using descripting operators

In [6]:
DT.shape # number of rows and columns

(5, 2)

In [7]:
DT.names # column names

('A', 'B')

In [8]:
DT.stypes # column types

(stype.int64, stype.float64)

 ## Data manipulation
